# Processo de Seleção de Variáveis

### Importando libs e funções:

Importando libs

In [0]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from matplotlib.colors import ListedColormap
from sklearn.linear_model import LinearRegression

Importando funções

In [0]:
# Função de escalonamento
def feature_scaling(data):
    sc = StandardScaler()
    return sc.fit_transform(data)

# Função que gera o gráfico dos resultados de regressão
def plot_results_linear(X, y, regressor, title):
    plt.scatter(X, y, color = 'red')
    plt.plot(X, regressor.predict(X), color = 'blue')
    plt.title(title)
    plt.xlabel('Tamanho do Lote')
    plt.ylabel('Preço de Vendas')
    plt.show()

# Função que gera o gráfico dos resultados de regerssão polinomial
def plot_results_poly(X, y, lin_reg_poly, poly_reg, title):
    plt.scatter(X, y, color = 'red')
    plt.plot(X, lin_reg_poly.predict(poly_reg.fit_transform(X)), color = 'blue')
    plt.title(title)
    plt.xlabel('Tamanho do Lote')
    plt.ylabel('Preço de Vendas')
    plt.show()    
    
# Função que gera o gráfico dos resultados de arvores
def plot_results_reg(X, y, regressor, title):     
    X_grid = np.arange(min(X), max(X), 0.01)
    X_grid = X_grid.reshape((len(X_grid), 1))
    plt.scatter(X, y, color = 'red')
    plt.plot(X_grid, regressor.predict(X_grid), color = 'blue')
    plt.title(title)
    plt.xlabel('Tamanho do Lote')
    plt.ylabel('Preço de Vendas')
    plt.show()
     

### Etapa de exploração e tratamento dos **dados**

Importando o dataset do nosso estudo. O objetivo dos modelos de regressão será de predizer o preço das casas de acordo com diferentes caracteristicas como: localização, área, etc.

Fonte: [Kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/r4phael/ml-course/master/data/pricing_houses.csv')

#Selecionando uma amostragem dos dados para uma melhor visualização
df = df.loc[:, ['LotArea', 'PoolArea', 'GarageArea', 'OverallCond','YearBuilt', 'MSZoning', 'SalePrice']]

Descrevendo o dataset

In [17]:
df.describe()

,LotArea,PoolArea,GarageArea,OverallCond,YearBuilt,SalePrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,10516.828082,2.758904,472.980137,5.575342,1971.267808,180921.195890
std,9981.264932,40.177307,213.804841,1.112799,30.202904,79442.502883
min,1300.000000,0.000000,0.000000,1.000000,1872.000000,34900.000000
25%,7553.500000,0.000000,334.500000,5.000000,1954.000000,129975.000000
50%,9478.500000,0.000000,480.000000,5.000000,1973.000000,163000.000000
75%,11601.500000,0.000000,576.000000,6.000000,2000.000000,214000.000000
max,215245.000000,738.000000,1418.000000,9.000000,2010.000000,755000.000000


Visualizando o dataset

In [18]:
df.head(5)

,LotArea,PoolArea,GarageArea,OverallCond,YearBuilt,MSZoning,SalePrice
0,8450,0,548,5,2003,RL,208500
1,9600,0,460,8,1976,RL,181500
2,11250,0,608,5,2001,RL,223500
3,9550,0,642,5,1915,RL,140000
4,14260,0,836,5,2000,RL,250000


Preenchendo os valores númericos nulos (NA) com a mediana.

In [19]:
df = df.fillna(df.median())

df.head(5)

,LotArea,PoolArea,GarageArea,OverallCond,YearBuilt,MSZoning,SalePrice
0,8450,0,548,5,2003,RL,208500
1,9600,0,460,8,1976,RL,181500
2,11250,0,608,5,2001,RL,223500
3,9550,0,642,5,1915,RL,140000
4,14260,0,836,5,2000,RL,250000


Definindo as variáveis independentes e dependentes

In [0]:
X = df.loc[:, 'LotArea'].values.reshape(-1,1)
y = df.loc[:, 'SalePrice'].values.reshape(-1,1)


## Forward Elimination


### Etapa de Seleção e Tratamento dos Dados

No processo de Forward Elimination, iremos selecionar as features incrementalmente uma por uma e analisamos se a mesma contribui para a melhoria do modelo. Porsteriormente, treinamos o modelo com a package OLS que realiza o processo de cálculo dos coeficientes para analise:

Selecionando a primeira feature: *LotArea*

In [21]:
df_forward = df.loc[:,['LotArea', 'SalePrice']]

df_forward.head(5)

,LotArea,SalePrice
0,8450,208500
1,9600,181500
2,11250,223500
3,9550,140000
4,14260,250000


### Realizando o Processo de Foward Elimination


Realizando o processo de forward elimination. Primeiro, será inserido uma coluna preenchida com valores 1 no começo da matriz de variáveis. Isso é realizada para que sejam feito os calculos necessários:

In [22]:
X = np.append(arr = np.ones((1460,1)).astype(int), values = df_forward, axis =1)

X[1:5,:]


array([[     1,   9600, 181500],
       [     1,  11250, 223500],
       [     1,   9550, 140000],
       [     1,  14260, 250000]])

Após isso, será utilizada a package OLS para calculo de importancia das features no output do modelo:

In [26]:
# Importando a package.
import statsmodels.regression.linear_model as sm

X_opt = X[:, [0,1]]
regressor_ols = sm.OLS(endog = y, exog = X[:, [0,1]]).fit()
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.070
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     109.1
Date:                Tue, 19 Nov 2019   Prob (F-statistic):           1.12e-24
Time:                        17:17:08   Log-Likelihood:                -18491.
No. Observations:                1460   AIC:                         3.699e+04
Df Residuals:                    1458   BIC:                         3.700e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.588e+05   2914.717     54.495      0.000    1.53e+05    1.65e+05
x1             2.1000      0.201     10.445      0.000       1.706       2.494
==============================================================================
Omnibus:                      587.660   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3374.003
Skew:                           1.788   Prob(JB):                         0.00
Kurtosis:                       9.532   Cond. No.                     2.11e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.11e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Analisando os valores acima, vimos que as features X1(*LotArea* - Área do lote) possui um P-value significativo, ou seja, dentro do level de signifcância definida (SL = .05) .Portanto, deixamos a mesma e escolhemos outra feature para incrementar no modelo conforme o processo de Forward Elimination.  

**Obs: Definimos um level de significância de .05 para que as features permaneçam no modelo (SL = .05).**

Adicionando a segunda feature no dataframe

In [27]:
df_forward = pd.concat([df_forward, df['GarageArea']], axis=1)

df_forward.head(5)

,LotArea,SalePrice,GarageArea
0,8450,208500,548
1,9600,181500,460
2,11250,223500,608
3,9550,140000,642
4,14260,250000,836


Calculando os coeficientes com a segunda features:

In [28]:

# Adicionando 1 na primeira coluna da matriz de features 
X = np.append(arr = np.ones((1460,1)).astype(int), values = df_forward, axis =1)

# Selecionando apenas as features de indice 0-const, 1-SalesPrice, 3-GarageArea
X_opt = X[:, [0,1,3]]
regressor_ols = sm.OLS(endog = y, exog = X_opt).fit()
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.412
Model:                            OLS   Adj. R-squared:                  0.412
Method:                 Least Squares   F-statistic:                     511.2
Date:                Tue, 19 Nov 2019   Prob (F-statistic):          6.34e-169
Time:                        17:17:39   Log-Likelihood:                -18156.
No. Observations:                1460   AIC:                         3.632e+04
Df Residuals:                    1457   BIC:                         3.633e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       6.322e+04   4015.973     15.742      0.000    5.53e+04    7.11e+04
x1             1.2453      0.163      7.663      0.000       0.927       1.564
x2           221.1574      7.587     29.151      0.000     206.276     236.039
==============================================================================
Omnibus:                      544.830   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5456.748
Skew:                           1.446   Prob(JB):                         0.00
Kurtosis:                      12.018   Cond. No.                     3.65e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.65e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**Final:** Todas as features acima estão dentro dentro intervalo de significância do modelo (SL = .05). Portanto, o processo continua de maneira incremental até que o modelo não seja improvisado com a adição de novas features.

Calculando o score do modelo:

In [31]:
regressor = LinearRegression()
regressor.fit(X_opt, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Analisando o novo score do modelo com a métrica r2

In [32]:
regressor.score(X_opt, y)

0.41235186539832547

##  Backward Elimination

### Etapa de Seleção e Tratamento dos Dados

Selecionando as principais features do dataset:

In [33]:
df = df.loc[:, ['LotArea', 'PoolArea', 'GarageArea', 'YearBuilt', 'SalePrice']]

df.head(5)

,LotArea,PoolArea,GarageArea,YearBuilt,SalePrice
0,8450,0,548,2003,208500
1,9600,0,460,1976,181500
2,11250,0,608,2001,223500
3,9550,0,642,1915,140000
4,14260,0,836,2000,250000


Definindo as variáveis indepedentes e dependentes, normalição das features e dividisão do dataset em conjunto de treinamento e testes:

In [0]:
X = df[df.columns[~df.columns.isin(['SalePrice'])]].values
y = df['SalePrice'].values.reshape(-1,1)

# Normalização das features:
X = feature_scaling(X)

# Dividindo os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

### Realizando o Processo de Backward Elimination


Realizando o processo de backward elimination. Primeiro, será inserido uma coluna preenchida com valores 1 no começo da matriz de variáveis. Isso é realizada para que sejam feito os calculos necessários:

In [35]:
X_train = np.append(arr = np.ones((1168,1)).astype(int), values = X_train, axis =1)

X_train[2,:]


array([ 1.        , -0.1743691 , -0.06869175, -2.21296298, -2.02923537])

Selecionando as variáveis do conjunto de treinamento e treinando o modelo com a package OLS que realiza o processo de cálculo dos coeficientes para analise:

In [37]:
# Importando a package.
import statsmodels.regression.linear_model as sm

X_opt = X_train[:, [0,1,2,3,4]]
regressor_ols = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.490
Model:                            OLS   Adj. R-squared:                  0.488
Method:                 Least Squares   F-statistic:                     279.4
Date:                Tue, 19 Nov 2019   Prob (F-statistic):          2.31e-168
Time:                        17:18:49   Log-Likelihood:                -14409.
No. Observations:                1168   AIC:                         2.883e+04
Df Residuals:                    1163   BIC:                         2.885e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.809e+05   1617.751    111.820      0.000    1.78e+05    1.84e+05
x1          1.268e+04   1533.794      8.267      0.000    9670.830    1.57e+04
x2          4729.7608   1570.186      3.012      0.003    1649.047    7810.475
x3          3.476e+04   1893.434     18.359      0.000     3.1e+04    3.85e+04
x4          2.331e+04   1807.127     12.897      0.000    1.98e+04    2.69e+04
==============================================================================
Omnibus:                      435.926   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3360.331
Skew:                           1.517   Prob(JB):                         0.00
Kurtosis:                      10.736   Cond. No.                         1.75
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Analisando os valores acima, vimos que a feature X2 (*PoolArea* - Área da piscina) possui o menor grau de importância, ou seja, o maior P-value, já que possui um p-value de .003, equanto as outras features possui um valor abaixo de .000 .Portanto, removemos a mesma e reiniciamos o processo conforme o algoritmo de Backward Elimination.  

**Obs: Definimos um level de significância de .03 nesse caso específico como exemplo para que as features permaneçam no modelo (SL = .05).**

In [38]:
#Analisando todas as features, exceto a feature X2 (Índice 1)

X_opt = X_train[:, [0,2,3,4]]
regressor_ols = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.460
Model:                            OLS   Adj. R-squared:                  0.459
Method:                 Least Squares   F-statistic:                     330.7
Date:                Tue, 19 Nov 2019   Prob (F-statistic):          2.90e-155
Time:                        17:18:50   Log-Likelihood:                -14443.
No. Observations:                1168   AIC:                         2.889e+04
Df Residuals:                    1164   BIC:                         2.891e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.811e+05   1663.777    108.821      0.000    1.78e+05    1.84e+05
x1          5649.3615   1610.913      3.507      0.000    2488.743    8809.980
x2          3.772e+04   1912.254     19.727      0.000     3.4e+04    4.15e+04
x3          2.214e+04   1853.006     11.949      0.000    1.85e+04    2.58e+04
==============================================================================
Omnibus:                      441.037   Durbin-Watson:                   1.983
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2831.186
Skew:                           1.602   Prob(JB):                         0.00
Kurtosis:                       9.922   Cond. No.                         1.67
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Final:** Todas as features acima estão dentro dentro intervalo de significância do modelo (SL = .05). Portanto, o processo é finalizado e seguimos para o treinamento do modelo.

In [39]:
regressor = LinearRegression()
regressor.fit(X_opt, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Analisando o novo score do modelo com a métrica r2

In [40]:
regressor.score(X_test, y_test)

0.4484569043113157

##  Bidirectional Elimination

### Etapa de Seleção e Tratamento dos Dados

Selecionando as principais features do dataset:

In [41]:
df = df.loc[:, ['LotArea', 'PoolArea', 'GarageArea', 'YearBuilt', 'SalePrice']]

df.head(5)

,LotArea,PoolArea,GarageArea,YearBuilt,SalePrice
0,8450,0,548,2003,208500
1,9600,0,460,1976,181500
2,11250,0,608,2001,223500
3,9550,0,642,1915,140000
4,14260,0,836,2000,250000


Definindo as variáveis indepedentes e dependentes, normalição das features e dividisão do dataset em conjunto de treinamento e testes:

In [0]:
X = df[df.columns[~df.columns.isin(['SalePrice'])]].values
y = df['SalePrice'].values.reshape(-1,1)

# Normalização das features:
X = feature_scaling(X)

# Dividindo os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

### Realizando o Processo de Bidirectional Elimination


Realizando o processo de Bidirectional elimination que é essencialmente um procedimento de Foward Elminiation, porém com a possibilidade de excluir uma variável selecionada em cada etapa como na  Backward Elimination, quando não há melhoria no modelo. Portanto, o processo é executado adicionando/removendo variáveis baseados em um critério especifico definido pelo usuário (p-values no nosso caso).

Primeiro, será adicionado uma variável constante como pré-requisito para execução dos procedimentos.

In [0]:
X_train = np.append(arr = np.ones((1168,1)).astype(int), values = X_train, axis =1)

Analisando as features da nossa matrix: 0 - Constante, 1 - LotArea, 2 - PoolArea, 3 - GarageArea, 4 - OverallCond, 5 - YearBuilt.

In [44]:
np.shape(X_train)

(1168, 5)

Selecionando as variáveis do conjunto de treinamento e treinando o modelo com a package OLS que realiza o processo de cálculo dos coeficientes para analise:

In [45]:
# Importando a package.
import statsmodels.regression.linear_model as sm

# Escolhendo algumas Features 
X_opt = X_train[:, [0,1,2,3,4]]
regressor_ols = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.490
Model:                            OLS   Adj. R-squared:                  0.488
Method:                 Least Squares   F-statistic:                     279.4
Date:                Tue, 19 Nov 2019   Prob (F-statistic):          2.31e-168
Time:                        17:18:50   Log-Likelihood:                -14409.
No. Observations:                1168   AIC:                         2.883e+04
Df Residuals:                    1163   BIC:                         2.885e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.809e+05   1617.751    111.820      0.000    1.78e+05    1.84e+05
x1          1.268e+04   1533.794      8.267      0.000    9670.830    1.57e+04
x2          4729.7608   1570.186      3.012      0.003    1649.047    7810.475
x3          3.476e+04   1893.434     18.359      0.000     3.1e+04    3.85e+04
x4          2.331e+04   1807.127     12.897      0.000    1.98e+04    2.69e+04
==============================================================================
Omnibus:                      435.926   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3360.331
Skew:                           1.517   Prob(JB):                         0.00
Kurtosis:                      10.736   Cond. No.                         1.75
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Analisando os valores acima, vimos que a feature X2 (*PoolArea* - Área da piscina) possui o menor grau de importância, ou seja, um P-value acima do threshold, já que possui um p-value de .003, equanto as outras features possui um valor abaixo de .001 .

Portanto, removemos essas features conforme o procedimento padrão. Porém, também iremos também adicionar as features restantes conforme o processo de Bidirectional Elimination.

**Obs: Definimos um level de significância de .05 para que as features permaneçam no modelo (SL = .03).**

In [46]:
#Analisando todas as features, exceto a feature X2 (Índice 1)

X_opt = X_train[:, [0,2,3,4]]
regressor_ols = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.460
Model:                            OLS   Adj. R-squared:                  0.459
Method:                 Least Squares   F-statistic:                     330.7
Date:                Tue, 19 Nov 2019   Prob (F-statistic):          2.90e-155
Time:                        17:18:50   Log-Likelihood:                -14443.
No. Observations:                1168   AIC:                         2.889e+04
Df Residuals:                    1164   BIC:                         2.891e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.811e+05   1663.777    108.821      0.000    1.78e+05    1.84e+05
x1          5649.3615   1610.913      3.507      0.000    2488.743    8809.980
x2          3.772e+04   1912.254     19.727      0.000     3.4e+04    4.15e+04
x3          2.214e+04   1853.006     11.949      0.000    1.85e+04    2.58e+04
==============================================================================
Omnibus:                      441.037   Durbin-Watson:                   1.983
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2831.186
Skew:                           1.602   Prob(JB):                         0.00
Kurtosis:                       9.922   Cond. No.                         1.67
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Procedimento Final:** Todas as features acima estão dentro dentro intervalo de significância do modelo (SL = .05). Portanto, o processo de Bidirectional Elimination é finalizado e seguimos para o treinamento do modelo.

In [47]:
regressor = LinearRegression()
regressor.fit(X_opt[:,1:], y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Analisando o novo score do modelo com a métrica r2, exceto com a feature X2 (Índice 0 no conjunto de testes)

In [48]:
regressor.score(X_test[:, [1,2,3]], y_test)

0.4484569043113157